In [190]:
from lxml import html
import requests
import sys
import pandas as pd
import numpy as np

class Tester(object):

    def __init__(self, name):
        self.name = str(name).upper().strip()
        
        #set up empty dataframe 
        self.titles = ['Time','Price','ShareVolume','']
        day_list = []
        #loop for different webpages/time frames (1-13)
        for j in range(1,14):
            #find number of pages for each time frame
            page = requests.get("http://www.nasdaq.com/symbol/" + self.name + '/time-sales?time=' + str(j))
            tree = html.fromstring(page.content)
            page_text = page.text
                    
            #find numbers from chart in html
            info_start = page_text.find('class="pagerlink"')
            info_end = page_text.find('" id="quotes_content_left_lb_LastPage"')
            info = page_text[info_start:info_end] 
            num = info[-2:]
            num = num.replace('=','')
                    
            #try while loop: while there is a next page? (1-(4:67)ish)
            #for i in range(1,2):
            for i in range(1,int(num)+1):
                
                try:
                    #get webpage in html content
                    self.page = requests.get("http://www.nasdaq.com/symbol/" + self.name + '/time-sales?time=' + str(j) +'&pageno=' + str(i))
                    self.tree = html.fromstring(self.page.content)
                    self.page_text = self.page.text
                    
                    #find numbers from chart in html
                    info_start = self.page_text.find('</thead>')
                    info_end = self.page_text.find('<div id="quotes_content_left__panelLowerNav')
                    self.info = self.page_text[info_start:info_end] 
                    
                    #strip of unnecessary spaces, etc
                    day = self.info.split("</tr>")
                    day_new = [days.replace("\r\n","") for days in day]
                    day1 =  [y.replace(" ","") for y in day_new]
                    day2 = [y.replace('<td>','') for y in day1]
                    day3 = [y.replace('</thead>','') for y in day2]
                    day4 = [y.replace('&nbsp;','') for y in day3]
                    day5 = [y.replace('<tr>','') for y in day4]
                    day6 = [y.replace('</div>','') for y in day5]
                    day7 = [y.replace('</table>','') for y in day6]
                    day8 = [y.replace('\t','') for y in day7]
                    
                    #split by day and add to dataframe
                    day_list += [x.split("</td>") for x in day8]
                    
                    self.data = pd.DataFrame(day_list,columns=self.titles)
                    
                except Exception as e:
                    print("ERROR:\n" + str(e.args) + "\n\nTerminating execution...")
                    sys.exit()
                    
        data = pd.DataFrame(day_list,columns=self.titles)
        self.data = data[['Time', 'Price']]
        
    def get_table(self):
        #get rid of whitespace row and column
        data = self.data.drop(self.data.index[len(self.data)-1])
        
        #strip numbers in dataframe of $, and get time frame correct
        data['Price'] = data['Price'].str.replace(",","")
        data['Price'] = data['Price'].str.strip('$').astype('float64') 
        
        #make time be each minute instead of by seconds
        data['Time'] = pd.to_datetime(data['Time'])
        data = data.set_index('Time')
        self.data = data.resample('T').mean()
        
        #np.savetxt('STAT485Day1.csv',data)
        
        return self.data
    
    
share=Tester('goog')
share.get_table()

,Price
Time,
2018-02-18 09:30:00,1088.505649
2018-02-18 09:31:00,1093.330868
2018-02-18 09:32:00,1090.111808
2018-02-18 09:33:00,1089.189895
2018-02-18 09:34:00,1090.479667
2018-02-18 09:35:00,1092.151250
2018-02-18 09:36:00,1094.003400
2018-02-18 09:37:00,1094.870950
2018-02-18 09:38:00,1096.058340
